In [20]:
#!/usr/bin/env python
# coding: utf-8

# ![](https://github.com/eraldoribeiro/assignment_robotarm2D/blob/main/robotArm01.png?raw=1)
# *Figure 1*: A two-dimensional articulated arm. The articulated structure has 3 local coordinate frames each one centered at a joint. For each part, the local x-axis is aligned with the part.

"""
Spheres and cylinders
"""
# (adapted from http://www.glowscript.org)
from vedo import *
import numpy as np

# Color of robot arm
robotColor = "green"
jointColor = "white"


def RotationMatrix(theta, axis_name):
    """ calculate single rotation of \(\theta\) matrix around x,y or z
        code from: https://programming-surgeon.com/en/euler-angle-python-en/
    input
        theta = rotation angle(degrees)
        axis_name = 'x', 'y' or 'z'
    output
        3x3 rotation matrix
    """

    c = np.cos(theta * np.pi / 180)
    s = np.sin(theta * np.pi / 180)
    if axis_name == 'x':
        rotation_matrix = np.array([[1, 0,  0],
                                    [0, c, -s],
                                    [0, s,  c]])
    if axis_name == 'y':
        rotation_matrix = np.array([[c,  0, s],
                                    [0,  1, 0],
                                    [-s,  0, c]])
    elif axis_name == 'z':
        rotation_matrix = np.array([[c, -s, 0],
                                    [s,  c, 0],
                                    [0,  0, 1]])
    return rotation_matrix


def createCoordinateFrameMesh():
    """Returns the mesh representing a coordinate frame
    Args:
      No input args
    Returns:
      F: vedo.mesh object (arrows for axis)

    """     

    _shaft_radius = 0.05
    _head_radius = 0.10
    _alpha = 1

    # x-axis as an arrow  
    x_axisArrow = Arrow(
                    start_pt=(0, 0, 0),
                    end_pt=(1, 0, 0),
                    s=None,
                    shaft_radius=_shaft_radius,
                    head_radius=_head_radius,
                    head_length=None,
                    res=12,
                    c='red',
                    alpha=_alpha
    )

    # y-axis as an arrow  
    y_axisArrow = Arrow(
                    start_pt=(0, 0, 0),
                    end_pt=(0, 1, 0),
                    s=None,
                    shaft_radius=_shaft_radius,
                    head_radius=_head_radius,
                    head_length=None,
                    res=12,
                    c='green',
                    alpha=_alpha
    )

    # z-axis as an arrow  
    z_axisArrow = Arrow(
                    start_pt=(0, 0, 0),
                    end_pt=(0, 0, 1),
                    s=None,
                    shaft_radius=_shaft_radius,
                    head_radius=_head_radius,
                    head_length=None,
                    res=12,
                    c='blue',
                    alpha=_alpha
    )

    originDot = Sphere(pos=[0,0,0], 
                       c="black", 
                       r=0.10)

    # Combine the axes together to form a frame
    F = x_axisArrow + y_axisArrow + z_axisArrow + originDot

    return F


def getLocalFrameMatrix(R_ij, t_ij): 
    """Returns the matrix representing the local frame
    Args:
      R_ij: rotation of Frame j w.r.t. Frame i 
      t_ij: translation of Frame j w.r.t. Frame i 
    Returns:
      T_ij: Matrix of Frame j w.r.t. Frame i. 

    """             
    # Rigid-body transformation [ R t ]
    T_ij = np.block([[R_ij,                t_ij],
                     [np.zeros((1, 3)),       1]])


    return T_ij


def forward_kinematics1(Phi, L1, L2, L3, L4):
    """Calculate the local-to-global frame matrices, 
    and the location of the end-effector.
    
    Args:
        Phi (4x1 nd.array): Array containing the four joint angles  
        L1, L2, L3, L4 (float): lengths of the parts
        
    Returns: 
        T_01, T_02, T_03, T_04: 4x4 nd.arrays, local-to-global matrices 
        e: 3x1 nd.array, location of end-effector 
    """

    R_01 = RotationMatrix(Phi[0], 'z')  
    R_12 = RotationMatrix(Phi[1], 'z')
    R_23 = RotationMatrix(Phi[2], 'z')
    R_34 = RotationMatrix(Phi[3], 'z')

    t_01 = np.array([[L1], [0], [0]])
    t_12 = np.array([[L2], [0], [0]])
    t_23 = np.array([[L2], [0], [0]]) 
    t_34 = np.array([[0], [0], [0]])

    T_01 = getLocalFrameMatrix(R_01, t_01) 
    T_12 = getLocalFrameMatrix(R_12, t_12)
    T_23 = getLocalFrameMatrix(R_23, t_23)
    T_34 = getLocalFrameMatrix(R_34, t_34)

    T_02 = T_01 @ T_12 
    T_03 = T_02 @ T_23
    T_04 = T_03 @ T_34

    e = T_04[0:3, 3] 
    
    return T_01, T_02, T_03, T_04, e


def forward_kinematics2(Phi, L1, L2, L3, L4):

    R_01 = RotationMatrix(Phi[0], 'z')  
    R_12 = RotationMatrix(Phi[1], 'z')
    R_23 = RotationMatrix(Phi[2], 'z')
    R_34 = RotationMatrix(Phi[3], 'z')

    t_01 = np.array([[0], [0], [0]])
    t_12 = np.array([[L1], [0], [0]])
    t_23 = np.array([[L2], [0], [0]]) 
    t_34 = np.array([[L3], [0], [0]])

    T_01 = getLocalFrameMatrix(R_01, t_01) 
    T_12 = getLocalFrameMatrix(R_12, t_12)
    T_23 = getLocalFrameMatrix(R_23, t_23)
    T_34 = getLocalFrameMatrix(R_34, t_34)

    T_02 = T_01 @ T_12 
    T_03 = T_02 @ T_23
    T_04 = T_03 @ T_34

    e = T_04[0:3, 3] 
    
    return T_01, T_02, T_03, T_04, e


def forward_kinematics3(Phi, L1, L2, L3, L4):

    
    R_01 = RotationMatrix(Phi[0], 'z')  
    R_12 = RotationMatrix(Phi[1], 'z')
    R_23 = RotationMatrix(Phi[2], 'z')
    R_34 = RotationMatrix(Phi[3], 'z')

    t_01 = np.array([[0], [0], [0]])
    t_12 = np.array([[L1], [0], [0]])
    t_23 = np.array([[L2], [0], [0]]) 
    t_34 = np.array([[L3], [0], [0]])

    T_01 = getLocalFrameMatrix(R_01, t_01) 
    T_12 = getLocalFrameMatrix(R_12, t_12)
    T_23 = getLocalFrameMatrix(R_23, t_23)
    T_34 = getLocalFrameMatrix(R_34, t_34)

    T_02 = T_01 @ T_12 
    T_03 = T_02 @ T_23
    T_04 = T_03 @ T_34

    e = T_04[0:3, 3] 
    
    return T_01, T_02, T_03, T_04, e


# Example of usage in the test cases:
L1, L2, L3, L4 = [5, 8, 3, 0]

# Test Case 1
Phi1 = np.array([30, -50, -30, 0])
T_01, T_02, T_03, T_04, e1 = forward_kinematics1(Phi1, L1, L2, L3, L4)
print("Test Case 1:")

print("e1:", e1)

# Test Case 2
Phi2 = np.array([0, 0, 0, 0])
T_01, T_02, T_03, T_04, e2 = forward_kinematics2(Phi2, L1, L2, L3, L4)
print("\nTest Case 2:")

print("e2:", e2)

# Test Case 3
Phi3 = np.array([-30, 50, 30, 0])
T_01, T_02, T_03, T_04, e3 = forward_kinematics3(Phi3, L1, L2, L3, L4)
print("\nTest Case 3:")

print("e3:", e3)


Test Case 1:
e1: [19.4457442   1.26383885  0.        ]

Test Case 2:
e2: [16.  0.  0.]

Test Case 3:
e3: [13.77603081  2.53429448  0.        ]
